## Load Emeddings

In [ ]:
import torch

def load_embedding_and_labels(accession, proc_dir=PROC_DIR):
    emb_path = proc_dir / f"{accession}_embedding.pt"
    labels_path = proc_dir / f"{accession}_labels.pt"
    embedding = torch.load(emb_path)
    labels = torch.load(labels_path)
    return embedding, labels

# Example usage for the first accession in df
example_accession = df["accession"].iloc[0]
embedding, labels = load_embedding_and_labels(example_accession)
print(f"Embedding shape: {embedding.shape}")
print(f"Labels shape: {labels.shape}")

Embedding shape: torch.Size([1, 1024])
Labels shape: torch.Size([340])


In [ ]:
%pip install graphpartpip install graphpart

TODO in Terminal!

<mmseqs createdb all_sequences.fasta all_sequences_db
mmseqs cluster all_sequences_db clusters tmp --min-seq-id 0.6 -c 0.8
mmseqs createtsv all_sequences_db all_sequences_db clusters clusters.tsv>

<conda install -c bioconda mmseqs2>

<graphpart split --input clusters.tsv --output splits.tsv --ratios 0.7 0.2 0.1>

In [ ]:
import pandas as pd

df = pd.read_csv("../data/results_with_sequence.csv")
with open("all_sequences.fasta", "w") as fasta:
    for i, row in df.iterrows():
        fasta.write(f">{row['accession']}\n{row['sequence']}\n")

In [ ]:
import pandas as pd

splits = pd.read_csv("splits.tsv", sep="\t", names=["accession", "split"])
split_dict = dict(zip(splits["accession"], splits["split"]))

4. Map splits back to your embeddings

In [ ]:
from pathlib import Path
import shutil

PROC_DIR = Path("../data/processed")
for split in ["train", "val", "test"]:
    (PROC_DIR / split).mkdir(exist_ok=True)

for accession, split in split_dict.items():
    emb_file = PROC_DIR / f"{accession}_embedding.pt"
    label_file = PROC_DIR / f"{accession}_labels.pt"
    if emb_file.exists():
        shutil.move(str(emb_file), str(PROC_DIR / split / f"{accession}_embedding.pt"))
    if label_file.exists():
        shutil.move(str(label_file), str(PROC_DIR / split / f"{accession}_labels.pt"))